# Task 1: Binary Represnetations


### Left Rotation 
`rotl` rotates the bits in a 32-bit unsigned integer `x` to the left by `n` places.

In [24]:
def rotl(x, n=1):

    # This is to ensure x is a 32-bit unsigned integer
    x = x & 0xFFFFFFFF 

    #This is to ensure n is within the range of 0 to 31
    n = n % 32
    
    return ((x << n) | (x >> (32 - n))) & 0xFFFFFFFF # Return the result of rotationg x left by n bits

### Example:

In [25]:
# Left rotation 
x = 5
print(format(x, '032b')) # 5 in 32 bits

# Rotate 5 to the left by 1
print(format(rotl(x), '032b'))

00000000000000000000000000000101
00000000000000000000000000001010


### Right Rotation
`rotr` rotates the bits in a 32-bit unsigned integer `x` to the right `n` places.

In [ ]:
def rotr(x, n=1):
    
    # This is to ensure x is a 32-bit unsigned integer
    x = x & 0xFFFFFFFF 

    #This is to ensure n is within the range of 0 to 31
    n = n % 32
    return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF # Return the result of rotationg x right

### Example:

In [23]:
x = 2
print(format(x, '032b')) # 2 in 32 bits

# Rotate 2 to the right by 1
print(format(rotr(x), '032b'))

00000000000000000000000000000010
00000000000000000000000000000001
